In [4]:
# !pip install accelerate

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
HF_TOKENS = os.getenv("HF_TOKENS")

In [7]:
from huggingface_hub import InferenceClient
api_key = ''
client = InferenceClient(api_key=HF_TOKENS)
messages = [
    {
        "role": "user",
        "content": "What is the capital of France?"
    }
]
completion = client.chat.completions.create(
    model="meta-llama/Llama-3.2-1B-Instruct",
    messages=messages,
    max_tokens=500
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_calls=None)


In [8]:
def chatbot_conversation():
    messages = [{"role": "user", "content": "Hello, how are you?"}]
    
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bye!")
            break
        
        messages.append({"role": "user", "content": user_input})
        
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.2-1B-Instruct",
            messages=messages,
            max_tokens=500
        )
        
        bot_reply = completion.choices[0].message['content']
        print(f"Bot: {bot_reply}")
        
        messages.append({"role": "assistant", "content": bot_reply})

chatbot_conversation()

Bye!


In [9]:
from huggingface_hub import InferenceClient
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Hugging Face API 키 입력
api_key = HF_TOKENS
client = InferenceClient(api_key=api_key)

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    """You are an analyst who answers questions accurately based on coin data and newspaper articles, English question/Korean question.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

def chatbot_conversation():
    
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bye!")
            break
        
        chat_history = "\n".join([msg['content'] for msg in messages])
        context = "Context information here"  
        
        prompt_input = prompt.format(chat_history=chat_history, question=user_input, context=context)
        
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.2-1B-Instruct",
            messages=[{"role": "user", "content": prompt_input}],
            max_tokens=500
        )
        
        bot_reply = completion.choices[0].message['content']
        print(f"Bot: {bot_reply}")
        
        messages.append({"role": "assistant", "content": bot_reply})

chatbot_conversation()


Bye!


In [38]:
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering, pipeline
import pandas as pd

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dsba-lab/koreapas-finetuned-korwikitq")
model = AutoModelForTableQuestionAnswering.from_pretrained("dsba-lab/koreapas-finetuned-korwikitq")

# Initialize the question answering pipeline
qa_model = pipeline('table-question-answering', model=model, tokenizer=tokenizer)

# Sample DataFrame for coin price data (no error-related columns)
df = pd.DataFrame({
    '코인': ['비트코인', '도지코인', '용희코인'],
    '가격': ['1억', '5000원', '0원']
})

def make_answer(q):
    if df.empty:
        return '데이터가 존재하지 않습니다.'
    
    answer = qa_model(query=q, table=df)

    if not answer['answer']:
        return '질문에 대한 답을 찾지 못했습니다.'
    
    if answer['coordinates'][0][-1] == 0:
        answer2 = answer['answer']
        return f"질문에 대한 답은 {answer2} 입니다."
    else:
        answer2 = answer['cells'][0]
        return f"그 코인의 가격은 {answer2} 입니다."

print(make_answer("adfe fse s efes se가격."))


질문에 대한 답은 비트코인, 도지코인 입니다.
